Run `pip install -r requirements.txt` first to install all dependencies.

In [1]:
!pip install gcsfs==2022.10.0
!pip install wandb==0.13.4
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.0 MB/s 
     |████████████████████████████████| 168 kB 50.4 MB/s 
     |████████████████████████████████| 182 kB 43.9 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 166 kB 58.8 MB/s 
     |████████████████████████████████| 166 kB 45.5 MB/s 
     |████████████████████████████████| 162 kB 45.5 MB/s 
     |████████████████████████████████| 162 kB 67.4 MB/s 
     |████████████████████████████████| 158 kB 46.7 MB/s 
     |████████████████████████████████| 157 kB 46.4 MB/s 
     |████████████████████████████████| 157 kB 51.2 MB/s 
     |████████████████████████████████| 157 kB 54.5 MB/s 
     |████████████████████████████████| 157 kB 50.4 MB/s 
     |████████████████████████████████| 157 kB 44.7

In [2]:
import gcsfs

import numpy as np
import pandas as pd

import wandb

from typing import List, Optional

import google.auth
from google.colab import auth

In [3]:
# connect to weights and biases
run = wandb.init(project="thesis", job_type="dataset-creation", entity="fbv")

dataset = wandb.Artifact(name="train_val_test", type="fe_log_bin_data")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
# connect to google cloud storage
auth.authenticate_user()
credentials, _ = google.auth.default()
fs = gcsfs.GCSFileSystem(project="thesis", token=credentials)

In [5]:
train = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/train_set_60.parquet",
    engine="fastparquet",
)
val = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/val_set_20.parquet",
    engine="fastparquet",
)
test = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/test_set_20.parquet",
    engine="fastparquet",
)


In [6]:
# unify for common preprocessing
X = pd.concat([train, val, test])

In [7]:
# isolate target
y = X[["buy_sell"]]
X = X.drop(["buy_sell"], axis=1)

In [8]:
# option features
X["time_to_maturity"] = (X["EXPIRATION"] - X["QUOTE_DATETIME"]).dt.days

In [9]:
# Trade features
mid = 0.5 * (X["ask_ex"] + X["bid_ex"])
X["midpoint_ex"] = mid

In [10]:
# apply positional encoding to dates
X["date_month_sin"] = np.sin(2 * np.pi * X["QUOTE_DATETIME"].dt.year / 12)
X["date_month_cos"] = np.cos(2 * np.pi * X["QUOTE_DATETIME"].dt.year / 12)

X["date_month_sin"] = np.sin(2 * np.pi * X["QUOTE_DATETIME"].dt.year / 12)
X["date_month_cos"] = np.cos(2 * np.pi * X["QUOTE_DATETIME"].dt.year / 12)

seconds_in_day = 24 * 60 * 60

seconds = (X["QUOTE_DATETIME"] - X["QUOTE_DATETIME"].dt.normalize()).dt.total_seconds()

X["date_time_sin"] = np.sin(2 * np.pi * seconds / seconds_in_day)
X["date_time_cos"] = np.cos(2 * np.pi * seconds / seconds_in_day)

# add year
X["date_year"] = (X["QUOTE_DATETIME"].dt.year - 2005) / (2017 - 2005)

In [11]:
feature_set_date = [
    "date_month_sin",
    "date_month_cos",
    "date_time_sin",
    "date_time_cos",
    "date_year",
]
feature_set_option = [
    "STRK_PRC",
    "ROOT",
    "time_to_maturity",
    "OPTION_TYPE",
]

feature_set_trade = [
    "TRADE_SIZE",
    "TRADE_PRICE",
    "BEST_ASK",
    "BEST_BID",
    "price_ex_lag",
    "price_ex_lead",
    "price_all_lag",
    "price_all_lead",
    "bid_ex",
    "ask_ex",
    "bid_size_ex",
    "ask_size_ex",    
    "midpoint_ex",
]

feature_set = [*feature_set_trade, *feature_set_date, *feature_set_option]
ignored_features = [x for x in X.columns.tolist() if x not in feature_set]
X.drop(columns=ignored_features, inplace=True)


In [12]:
# log transform
log_columns = [
    "TRADE_PRICE",
    "STRK_PRC",
    "BEST_ASK",
    "BEST_BID",
    "price_ex_lag",
    "price_ex_lead",
    "price_all_lag",
    "price_all_lead",
    "bid_ex",
    "ask_ex",
    "midpoint_ex",
    "TRADE_SIZE",
    "bid_size_ex",
    "ask_size_ex",
]

# + 1 to prevent inf
X[log_columns] = np.log(X[log_columns] + 1)


In [13]:
# binarize

# select categorical e. g., option type and strings e. g., ticker
cat_columns = X.select_dtypes(include=["category", "object"]).columns.tolist()
print(cat_columns)

# binarize categorical similar to Borisov et al.
X[cat_columns] = X[cat_columns].apply(lambda x: pd.factorize(x)[0])


['ROOT', 'OPTION_TYPE']


In [14]:
# treat inf as nan
X.replace([np.inf, -np.inf], np.nan, inplace=True)

In [15]:
X["buy_sell"] = y

In [16]:
# separate again for training scaling
X_train = X.loc[train.index, :]
X_val = X.loc[val.index, :]
X_test = X.loc[test.index, :]

y_train = y.loc[train.index, :]
y_val = y.loc[val.index, :]
y_test = y.loc[test.index, :]

In [17]:
output_path = f"gs://thesis-bucket-option-trade-classification/data/fe_log_bin_data/train_set_60.parquet"
train.to_parquet(output_path)
dataset.add_reference(output_path,name='train_set_60')

[<ManifestEntry ref: gs://thesis-bucket-option-trade-classification/data/fe_log_bin_data/train_set_60.parquet/train_set_60>]

In [18]:
output_path = f"gs://thesis-bucket-option-trade-classification/data/fe_log_bin_data/val_set_20.parquet"
val.to_parquet(output_path)
dataset.add_reference(output_path,name='val_set_20')

[<ManifestEntry ref: gs://thesis-bucket-option-trade-classification/data/fe_log_bin_data/val_set_20.parquet/val_set_20>]

In [19]:
output_path = f"gs://thesis-bucket-option-trade-classification/data/fe_log_bin_data/test_set_20.parquet"
test.to_parquet(output_path)
dataset.add_reference(output_path,name='test_set_20')

[<ManifestEntry ref: gs://thesis-bucket-option-trade-classification/data/fe_log_bin_data/test_set_20.parquet/test_set_20>]

In [20]:
run.finish()